# 第3章 中文分词技术

## 3.1 中文分词简介
中文自动分词主要归纳为：

+ 规则分词。最早兴起的方法，主要是通过人工设立词库，按照一定方式进行匹配切分，其实现简单高效，但是对新词很难进行处理

+ 统计分词。能够较好的应对新词发现等特殊场景，但是，太过于依赖语料的质量

+ 混合分词。

## 3.2 规则分词

基于规则的分词是一种机械分词方法，主要是通过维护词典，在切分语句时，将语句的每个字符串与词表中的词进行逐一匹配，找到则切分，否则不予以切分。

按照匹配切分的方式，主要有：

+ 正向最大匹配法

+ 逆向最大匹配法

+ 双向最大匹配法

### 3.2.1 正向最大匹配法（Maximum Match Method，MM法）

假定分词词典中的最长词有i个汉字字符，则用被处理文档的当前字串中的前i个字作为匹配字段，查找字典。若字典中存在这样的一个i字词，则匹配成功，匹配字段被作为一个词切分出来。如果词典中找不到这样的一个i字词，则匹配失败，将匹配字段中的最后一个字去掉，对剩下的字串重新进行匹配处理。如此进行下去，直到匹配成功，即切分出一个词或剩余字串的长度为零为止。这样就完成了一轮匹配，然后取下一个i字字串进行匹配处理，直到文档被扫描完为止。

### 3.2.2 逆向最大匹配法（Reverse Maximum Match Method， RMM法）

基本原理和MM法相同，不同的是分词切分的方向与MM法相反，从被处理文档的末端开始匹配扫描。

由于汉语中偏正结果较多，若从后向前匹配，可以适当提高精确度。统计结果表明，单纯使用正向最大匹配的错误率为1/169，单纯使用逆向最大匹配的错误率为1/245。

### 3.2.3 双向最大匹配法（Bi-directction Matching method）

将正向最大匹配法得到的分词结果和逆向最大匹配法得到的结果进行比较，然后按照最大匹配原则，选取词数切分最少的作为结果。


In [3]:
# 逆向最大匹配
class IMM(object):
    def __init__(self, dic_path):
        self.dictionary = set()
        self.maximum = 0
        # 读取词典
        with open(dic_path, 'r', encoding= 'utf8') as f:
            for line in f:
                line = line.strip()
                if not line:
                    continue
                self.dictionary.add(line)
                self.maximum = max(len(line),self.maximum)
    def cut(self, text):
        result = []
        index = len(text)
        while index > 0:
            word = None
            for size in range(self.maximum, 0, -1):  #9，8，7
                if index - size < 0:
                    continue
                piece = text[(index - size):index]
                if piece in self.dictionary:
                    word = piece
                    result.append(word)
                    index -= size
                    break
            if word is None:
                index -= 1
        return result[::-1]
def main():
    text = "南京市长江大桥"
    tokenizer = IMM('./data/imm_dic.utf8')
    print(tokenizer.cut(text))

main()        

['南京市', '长江大桥']


In [1]:
# 正向最大匹配
class MM(object):
    def __init__(self, dic_path):
        self.dictionary = set()
        self.maximun = 0
        with open(dic_path, 'r', encoding= 'utf8') as f:
            for line in f:
                line = line.strip()
                if not line:
                    continue
                self.dictionary.add(line)
                self.maximun = max(len(line), self.maximun)
    def cut(self, text):
        result = []
        index = len(text)
        str = 0 
        size = 0
        while str + size -1  < 80:
            word = None
            for size in range(self.maximun , 0, -1):
                if index - size < 0:
                    continue
                piece = text[str:str+size]
                if piece in self.dictionary:
                    word = piece
                    result.append(word)
                    str += size
                    break
            if word is None:
                str += 1
        return result[::]
def mi():
    text = "南京市长江大桥"
    tokenizer = MM('data\imm_dic.utf8')
    print(tokenizer.cut(text))

mi()        


['南京市长', '大桥']


## 3.3 统计分词

统计语料中相邻共现的各个字的组合的频度，当组合频度高于某一个临界值时，我们便可认为字组可能会构成一个词语。

基于统计的分词，一般要做如下两步操作：

1）建立统计语言模型

2）对句子进行单词划分，然后对划分结果进行概率计算，获得概率最大的分词方式。这里就用到了统计学校算法，如隐含马尔可夫（HMM)、条件随机场（CRF）等。

### 3.3.1 语言模型



用概率论的专业术语描述语言模型就是：为长度为m的字符串确定其概率分布$P(\omega_1, \omega_2, \dots, \omega_m)$，其中$\omega_1$到$\omega_m$依次表示文本中的各个词语。一般采用链式法则计算其概率值，如下式所示：


$$
P(\omega_1, \omega_2, \dots, \omega_m) = P(\omega_1)P(\omega_2|\omega_1)P(\omega_3|\omega_1,\omega_2)\\
\dots P(\omega_i|\omega_1,\omega_2, \dots, \omega_(i-1))\dots P(\omega_m|\omega_1, \omega_2, \dots, \omega_(m-1))
$$


简单地说，语言模型就是用来计算一个句子的概率的模型，也就是判断一句话是否是人话的概率。

公式从第三项起的每一项计算难度都很大。为解决该问题，有人提出n元模型（n-gram mmodel）降低该计算难度。_
所谓n元模型就是在估算条件概率时，忽略距离大于等于n的上文词的影响，所以可以简化为：

$$ P(\omega_i|\omega_1, \omega_2, \dots, \omega_(i-1)) \approx P(\omega_i|\omega_1, \omega_(i-(n-1)), \dots, \omega_(i-1)) \tag{3.2}$$

当 n = 1时，称为一元模型（unigram medel），此时整个句子的概率可表示为：$P(\omega_1, \omega_2, \dots, \omega_m) = P(\omega_1)P(\omega_2)\dots P(\omega_m)$，即在一元语言模型中，整个句子的概率等于各个词语概率的乘积，即各个词之间都是相互独立的，损失了句中的语序信息。

当 n = 2时，称为二元模型（bigram model），式（3.2）变为$P(\omega_i|\omega_1, \omega_2, \dots, \omega_(i-1)) = P(\omega_i|\omega_(i-1))$

当 n = 3时，称为三元模型（trigram model），式（3.2）变为$P(\omega_i|\omega_1, \omega_2, \dots, \omega_(i-1)) = P(\omega_i|\omega_(i-2), \omega_(i-1))$

*https://zhuanlan.zhihu.com/p/28080127*

显然当n $\geq$ 2时，该模型是可以保留一定的词序信息的，而且n越大，保留的词序信息约丰富，但计算成本也呈指数级增长。

一般使用频率计数的比例来计算n元条件概率，如式（3.3）所示：





$$ P(\omega_i|\omega_(i-(n-1)), \dots, \omega_(i-1)) = \frac{count(\omega_(i-(n-1)), \omega_(i-1), \omega_i)}{count(\omega_(i-(n-1)),\dots, \omega_(i-1))} \tag{3.3}$$

式子中$count(\omega_(i-(n-1)), \dots, \omega_(i-1))$表示词语$\omega_(i-(n-1)), \dots, \omega_(i-1)$在语料库中出现的总次数。

n越大，模型包含的词序信息约丰富，计算量也越大，同时，长度越长的文本序列出现的次数也会减少。

## 3.4 混合分词

事实上，目前不管是基于规则的算法、还是基于HMM、CRF或者deep learning等的方法，其分词效果在具体任务中，差距并没有那么明显。在实际应用中，最常用的方式就是先基于词典的方式进行分词，然后再用统计分词方法进行辅助。


## 3.5 中文分词工具——jieba
jieba分词官网地址是*https://github.com/fxsjy/jieba*

jiaba字典和其他常用字典*https://github.com/fighting41love/funNLP*

### 3.5.1 jieba的三种分词模式

+ 精确模式：试图将句子最精确地切开，适合文本分析

+ 全模式：把句子中所以可以成词的词语都扫描出来，速度非常快，但是不能解决歧义。

+ 搜索引擎模式：在精确模式的基础上，对长词再次切分，提高召回率，适合用于搜索引擎分词。

可使用 jieba.cut 和 jieba.cut_for_search 方法进行分词，两者所返回的结构都是一个可迭代的 generator，或者直接使用 jieba.lcut 以及 jieba.lcut_for_search 直接返回 list。



In [1]:
import jieba 
sent = '中文分词是文本处理不可缺少的一步！'
seg_list = jieba.cut(sent, cut_all = True)
print('全模式：', '/'.join(seg_list))
type(seg_list)

seg_llist = jieba.lcut(sent, cut_all = True)
print('全模式(返回列表): {0}'.format(seg_llist))


seg_list = jieba.cut(sent, cut_all= False)
print('精确模式：', '/'.join(seg_list))

seg_list = jieba.cut(sent)
print('默认精确模式：', '/'.join(seg_list))

seg_list = jieba.cut_for_search(sent)
print('搜索引擎模式：', '/'.join(seg_list))

Building prefix dict from the default dictionary ...
Loading model from cache C:\Users\62318\AppData\Local\Temp\jieba.cache
Loading model cost 1.339 seconds.
Prefix dict has been built successfully.
全模式： 中文/分词/是/文本/文本处理/本处/处理/不可/缺少/的/一步/！
全模式(返回列表): ['中文', '分词', '是', '文本', '文本处理', '本处', '处理', '不可', '缺少', '的', '一步', '！']
精确模式： 中文/分词/是/文本处理/不可/缺少/的/一步/！
默认精确模式： 中文/分词/是/文本处理/不可/缺少/的/一步/！
搜索引擎模式： 中文/分词/是/文本/本处/处理/文本处理/不可/缺少/的/一步/！


在 jieba 中，对于未登录到词库的词，使用了基于汉字成词能力的 HMM 模型和 Viterbi 算法，其大致原理是：采用四个隐含状态，分别表示为单字成词，词组的开头，词组的中间，词组的结尾。通过标注好的分词训练集，可以得到 HMM 的各个参数，然后使用 Viterbi 算法来解释测试集，得到分词结果。

### 3.5.2 添加自定义词典

开发者可以指定自定义词典，以便包含 jieba 词库里没有的词，词典格式如下：

词语 词频（可省略） 词性（可省略）

例如：

创新办 3 i

云计算 5

凱特琳 nz

虽然 jieba 有新词识别能力，但自行添加新词可以保证更高的正确率














### 3.5.3 载入词典

使用 jieba.load_userdict(file_name) 即可载入词典。





In [6]:
#示例 文本
sample_text = "周大福是创新办主任也是云计算方面的专家"
# 未加载词典
print("【未加载词典】：" + '/ '.join(jieba.cut(sample_text)))

# 载入词典
jieba.load_userdict("./userdict.txt")

# 加载词典后
print("【加载词典后】：" + '/ '.join(jieba.cut(sample_text)))



【未加载词典】：周大福/ 是/ 创新/ 办/ 主任/ 也/ 是/ 云/ 计算/ 方面/ 的/ 专家
【加载词典后】：周大福/ 是/ 创新办/ 主任/ 也/ 是/ 云计算/ 方面/ 的/ 专家


##### 3.5.4 调整词典

使用 add_word(word, freq=None, tag=None) 和 del_word(word) 可在程序中动态修改词典。


In [8]:
jieba.add_word('石墨烯') #增加自定义词语
jieba.add_word('凱特琳', freq=42, tag='nz') #设置词频和词性 
jieba.del_word('自定义词') #删除自定义词语 
# 调节词频前
print("【调节词频前】：" + '/'.join(jieba.cut('如果放到post中将出错。', HMM=False)))

# 调节词频
jieba.suggest_freq(('中', '将'), True)
# 调节词频后
print("【调节词频后】：" + '/'.join(jieba.cut('如果放到post中将出错。', HMM=False)))

【调节词频前】：如果/放到/post/中将/出错/。
【调节词频后】：如果/放到/post/中/将/出错/。


### 3.5.5 关键词提取
jieba 提供了两种关键词提取方法，分别基于 TF-IDF 算法和 TextRank 算法。


#### 3.5.5.1基于 TF-IDF 算法的关键词提取
TF-IDF(Term Frequency-Inverse Document Frequency, 词频-逆文件频率)是一种统计方法，用以评估一个词语对于一个文件集或一个语料库中的一份文件的重要程度，其原理可概括为：

一个词语在一篇文章中出现次数越多，同时在所有文档中出现次数越少，越能够代表该文章。

计算公式：TF-IDF = TF * IDF，其中：

+ TF(term frequency, TF)：词频，某一个给定的词语在该文件中出现的次数，计算公式：$$ TF_\omega = \frac{在某类中词条\omega 出现的次数}{该类所有的词条数目}$$或$$ TF = \frac{某个词在文章中出现次数}{文章的总词数}$$

+ IDF(inverse document frequency, IDF)：逆文件频率，如果包含词条的文件越少，则说明词条具有很好的类别区分能力，计算公式：$$ IDF = log(\frac{语料库的文档总数}{包含词条\omega的文档数+1}) $$

如果一个词越常见，那么分母就越大，逆文档频率就越小越接近0。分母之所以要加1，是为了避免分母为0（即所有文档都不包含该词）








通过 jieba.analyse.extract_tags 方法可以基于 TF-IDF 算法进行关键词提取，该方法共有 4 个参数：

+ sentence：为待提取的文本

+ topK：根据tf-idf值对词频词典中的词进行降序排序，然后输出topK个词作为关键词，默认值为 20

+ withWeight：是否一并返回关键词权重值，默认值为 False

+ allowPOS：仅包括指定词性的词，默认值为空

#### 3.5.5.2 词性对照表（allowPOS可选值）
##### 名词 (1个一类，7个二类，5个三类)
名词分为以下子类：
n 名词
nr 人名
nr1 汉语姓氏
nr2 汉语名字
nrj 日语人名
nrf 音译人名
ns 地名
nsf 音译地名
nt 机构团体名
nz 其它专名
nl 名词性惯用语
ng 名词性语素

##### 时间词(1个一类，1个二类)
t 时间词
tg 时间词性语素

##### 处所词(1个一类)
s 处所词

##### 方位词(1个一类)
f 方位词

##### 动词(1个一类，9个二类)
v 动词
vd 副动词
vn 名动词
vshi 动词“是”
vyou 动词“有”
vf 趋向动词
vx 形式动词
vi 不及物动词（内动词）
vl 动词性惯用语
vg 动词性语素

##### 形容词(1个一类，4个二类)
a 形容词
ad 副形词
an 名形词
ag 形容词性语素
al 形容词性惯用语

#####  区别词(1个一类，2个二类)
b 区别词
bl 区别词性惯用语

##### 状态词(1个一类)
z 状态词

##### 代词(1个一类，4个二类，6个三类)
r 代词
rr 人称代词
rz 指示代词
rzt 时间指示代词
rzs 处所指示代词
rzv 谓词性指示代词
ry 疑问代词
ryt 时间疑问代词
rys 处所疑问代词
ryv 谓词性疑问代词
rg 代词性语素

##### 数词(1个一类，1个二类)
m 数词
mq 数量词

##### 量词(1个一类，2个二类)
q 量词
qv 动量词
qt 时量词

##### 副词(1个一类)
d 副词

##### 介词(1个一类，2个二类)
p 介词
pba 介词“把”
pbei 介词“被”

##### 连词(1个一类，1个二类)
c 连词
cc 并列连词

##### 助词(1个一类，15个二类)
u 助词
uzhe 着
ule 了 喽
uguo 过
ude1 的 底
ude2 地
ude3 得
usuo 所
udeng 等 等等 云云
uyy 一样 一般 似的 般
udh 的话
uls 来讲 来说 而言 说来
uzhi 之
ulian 连 （“连小学生都会”）

##### 叹词(1个一类)
e 叹词

##### 语气词(1个一类)
y 语气词(delete yg)

##### 拟声词(1个一类)
o 拟声词
前缀(1个一类)

h 前缀
##### 后缀(1个一类)
k 后缀

##### 字符串(1个一类，2个二类)
x 字符串
xx 非语素字
xu 网址URL

##### 标点符号(1个一类，16个二类)
w 标点符号
wkz 左括号，全角：（ 〔 ［ ｛ 《 【 〖 〈 半角：( \[ { <
wky 右括号，全角：） 〕 ］ ｝ 》 】 〗 〉 半角： ) \] { >
wyz 左引号，全角：“ ‘ 『
wyy 右引号，全角：” ’ 』
wj 句号，全角：。
ww 问号，全角：？ 半角：?
wt 叹号，全角：！ 半角：!
wd 逗号，全角：， 半角：,
wf 分号，全角：； 半角： ;
wn 顿号，全角：、
wm 冒号，全角：： 半角： :
ws 省略号，全角：…… …
wp 破折号，全角：—— －－ ——－ 半角：— —-
wb 百分号千分号，全角：％ ‰ 半角：%
wh 单位符号，全角：￥ ＄ ￡ ° ℃ 半角：$

版权声明：本文为CSDN博主「apriaaaa」的原创文章，遵循 CC 4.0 BY-SA 版权协议

原文链接：*https://blog.csdn.net/apriaaaa/article/details/90261723*


In [11]:
import jieba

import jieba.analyse as anls
s = "此外，公司拟对全资子公司吉林欧亚置业有限公司增资4.3亿元，增资后，吉林欧亚置业注册资本由7000万元增加到5亿元。吉林欧亚置业主要经营范围为房地产开发及百货零售等业务。目前在建吉林欧亚城市商业综合体项目。2013年，实现营业收入0万元，实现净利润-139.13万元。"

for x, w in anls.extract_tags(s, topK=20, withWeight=True):
    print('%s %s' % (x, w))



欧亚 0.7300142700289363
吉林 0.659038184373617
置业 0.4887134522112766
万元 0.3392722481859574
增资 0.33582401985234045
4.3 0.25435675538085106
7000 0.25435675538085106
2013 0.25435675538085106
139.13 0.25435675538085106
实现 0.19900979900382978
综合体 0.19480309624702127
经营范围 0.19389757253595744
亿元 0.1914421623587234
在建 0.17541884768425534
全资 0.17180164988510638
注册资本 0.1712441526
百货 0.16734460041382979
零售 0.1475057117057447
子公司 0.14596045237787234
营业 0.13920178509021275


#### 3.5.5.3 基于 TextRank 算法的关键词提取

TextRank 是另一种关键词提取算法，基于大名鼎鼎的 PageRank，以词为节点，以共现关系建立起节点之间的链接。TextRank实际上是依据位置与词频来计算词的权重的。

通过 jieba.analyse.textrank 方法可以使用基于 TextRank 算法的关键词提取，其与 'jieba.analyse.extract_tags' 有一样的参数，但前者默认过滤词性（allowPOS=('ns', 'n', 'vn', 'v')）。


##### PageRank算法
PageRank可以定义在任意有向图上，后来被应用到社会影响力分析、文本摘要等多个问题。

###### 基本想法：

在有向图上定义一个随机游走模型，即一阶马尔可夫链，描述随机游走者沿着有向图随机访问各个结点的行为。

在一定条件下，极限情况访问每个结点的概率收敛到平稳分布，各个结点的平稳分布概率值就是其PageRank值，表示结点的重要度。

PageRank值越高，网页就越重要。

假设浏览者在每个网页依照连接出去的超链接以等概率跳转到下一个网页，并在网上持续不断进行这样的随机跳转，这个过程形成一阶马尔可夫链。

![Markdown](http://i1.fuimg.com/611786/054bef7100fd2126.png)

![Markdown](http://i1.fuimg.com/611786/5f84e2e696ccecaf.png)

图片来源：*https://blog.csdn.net/qq_38842357/article/details/80872480*



先假设一个初始分布，通过迭代，不断计算所有网页的PageRank值，直到收敛为止。

在有向图上的随机游走形成马尔可夫链。即下一个时刻的状态只依赖于当前的状态，与过去无关。

###### PageRank的核心思想：

+ 如果一个网页被很多其他网页链接到的话说明这个网页比较重要，也就是PageRank值会相对较高

+ 如果一个PageRank值很高的网页链接到一个其他的网页，那么被链接到的网页的PageRank值会相应地因此而提高






$$ S(V_i) = (1 - d) + d * \sum_{j \varepsilon In(V_i)} \frac{1}{|Out(V_j)|} S(V_j) $$

In(Vi)表示网页Vi的所有入链的集合Vi表示某个网页，Vj表示链接到Vi的网页（即Vi的入链），S(Vi)表示网页Vi的PR值，Out(Vj)表示$V_j$指向的链接，d表示阻尼系数，是用来克服这个公式中“d *”后面的部分的固有缺陷用的：如果仅仅有求和的部分，那么该公式将无法处理没有入链的网页的PR值，因为这时，根据该公式这些网页的PR值为0，但实际情况却不是这样，根据实验的结果，在0.85的阻尼系数下，大约100多次迭代PR值就能收敛到一个稳定的值，而当阻尼系数接近1时，需要的迭代次数会陡然增加很多，且排序不稳定。


###### TextRank算法的基本思想
将文档看作一个词的网络，该网络中的链接表示词与词之间的语义关系。以词为节点，以共现关系建立起节点之间的链接，需要强调的是，PageRank中是有向边，而TextRank中是无向边，或者说是双向边

$$ S(V_i) = (1 - d) + d * \sum_{j \varepsilon In(V_i)} \frac{\omega_{ij}}{\sum_{V_j \varepsilon Out(V_j) }\omega_{ij} } WS(V_j) $$

可以看出，该公式仅仅比PageRank多了一个权重项Wji，用来表示两个节点之间的边连接有不同的重要程度。

###### TextRank用于关键词提取的算法如下:

1. 把给定的文本T按照完整句子进行分割，$T=[S_1,S_2,......,S_m]$，

2. 对于每个句子，进行分词和词性标注处理，并过滤掉停用词，只保留指定词性的单词，如名词、动词、形容词，即，Si=$[pi_1，pi_2，...，pi_n]$

3. 构建候选关键词图G = (V,E)，其中V为节点集，由2生成的候选关键词组成，然后采用共现关系（co-occurrence）构造任两点之间的边，两个节点之间存在边仅当它们对应的词汇在长度为K的窗口中共现，即：$[p_1,p_2,...,p_k][p_2,p_3,...,p_{k+1}]$等都是一个个的窗口，在一个窗口中如果两个单词同时出现，则认为对应单词节点间存在一个边，

4. 根据PageRank原理中的衡量重要性的公式，初始化各节点的权重，然后迭代计算各节点的权重，直至收敛，

5. 对节点权重进行倒序排序，从而得到最重要的T个单词，作为候选关键词，

6. 由（5）得到最重要的T个单词，在原始文本中进行标记，若形成相邻词组，则组合成多词关键词。例如，文本中有句子“Matlab code for plotting ambiguity function”，如果“Matlab”和“code”均属于候选关键词，则组合成“Matlab code”加入关键词序列。

 例如：

 \['有','媒体', '曝光','高圆圆', '和', '赵又廷','现身', '台北', '桃园','机场','的', '照片'\]

 对于‘媒体‘这个单词，就有（'媒体', '曝光'）、（'媒体', '圆'）、（'媒体', '和'）、（'媒体', '赵又廷'）4条边，且每条边权值为1，当这条边在之后再次出现时，权值再在基础上加1。

 有了这些数据后，我们就可以构建出候选关键词图G = (V,E)

 



In [12]:
for x, w in anls.textrank(s, withWeight=True):
    print('%s %s' % (x, w))

吉林 1.0
欧亚 0.9966893354178172
置业 0.6434360313092776
实现 0.5898606692859626
收入 0.43677859947991454
增资 0.4099900531283276
子公司 0.35678295947672795
城市 0.34971383667403655
商业 0.34817220716026936
业务 0.3092230992619838
在建 0.3077929164033088
营业 0.3035777049319588
全资 0.303540981053475
综合体 0.29580869172394825
注册资本 0.29000519464085045
有限公司 0.2807830798576574
零售 0.27883620861218145
百货 0.2781657628445476
开发 0.2693488779295851
经营范围 0.2642762173558316


### 3.5.6 自定义语料库
关键词提取所使用逆向文件频率（IDF）文本语料库和停止词（Stop Words）文本语料库可以切换成自定义语料库的路径。



In [15]:
jieba.analyse.set_stop_words("userdict.txt")
jieba.analyse.set_idf_path("idf.txt.big")
for x, w in anls.extract_tags(s, topK=20, withWeight=True):
    print('%s %s' % (x, w))

吉林 1.0174270215234043
欧亚 1.0174270215234043
置业 0.7630702661425532
万元 0.7630702661425532
增资 0.5087135107617021
亿元 0.5087135107617021
实现 0.5087135107617021
此外 0.25435675538085106
公司 0.25435675538085106
全资 0.25435675538085106
子公司 0.25435675538085106
有限公司 0.25435675538085106
4.3 0.25435675538085106
注册资本 0.25435675538085106
7000 0.25435675538085106
增加 0.25435675538085106
主要 0.25435675538085106
经营范围 0.25435675538085106
房地产 0.25435675538085106
开发 0.25435675538085106


### 3.5.7 词性标注

jieba.posseg.POSTokenizer(tokenizer=None) 新建自定义分词器。

+ tokenizer 参数可指定内部使用的 jieba.Tokenizer 分词器。jieba.posseg.dt 为默认词性标注分词器。



In [18]:
import jieba.posseg as pseg
words = pseg.cut("他改变了中国")
for word, flag in words:
    print("{0} {1}".format(word, flag))

他 r
改变 v
了 ul
中国 ns
